In [1]:
import numpy as np
import scipy as sp
from scipy.optimize import minimize
from scipy.optimize import Bounds

# Mathematical Background

We model an Investment Portfolio as follows:

$$ \omega = \left( \omega_{1} , \omega_{2}, ... , \omega_{N}\right) $$

With constraint

$$ \sum_{i = 1}^{N} \omega_{i} = 1 $$

Such that our investment $I = $ £ $X$ punds becomes diversified into

$$ I = X\omega_{1} + X\omega_{2} + ... + X\omega_{N} $$

The return on an investment is the sum of all individual returns:

$$ r = \sum_{i = 1}^{N} r_i \omega_{i} $$

Multiply this by $X$ from $I =$ £ $X$ and we find the return on our invested pounds

Expected Returns: We denote the expected returns on an investment (before we make the trade) as $\mu_{i}$ and therefore the total expected return as 

$$ \mu  = \sum_{i = 1}^{N} \mu_{i} \omega_{i} $$ 

## The Constrants:

What it does not do: This model tells you nothing of the expected returns $\mu$ or how to calculate them. This must be done through assumptions from outside of the model.

What it does do: This model calculates $\omega$ distribution via just one method of maximizing the entropy based on some constraints and expected returns $\mu$ as imputs.

Input:

$$ In = \mu$$

Constraints:

1. Maximise Entropy: $$ E(\omega) = \sum_{i = 1}^{N} - \omega_{i} \ln \omega_{i} $$

2. Maximise Total Returns: $$ \mu  = \sum_{i = 1}^{N} \mu_{i} \omega_{i} $$ 

3. Unit Sum of Portfolio: $$ \sum_{i = 1}^{N} \omega_{i} = 1 $$

Output:

$$ Out = \omega$$

## Rational idea behind the model:

Given a distribution of expected returns on investments, each with their own risks and uncertainties, there are many probability distributions for any set of these. We would like to find what is the distribution $\omega$ based on this expected returns $\mu$ for which maximizes the amount of unertainty. In other words, it is the distribution least committed to the information not given to us. And therefore, it is the distribution with the least amount of bias in the model.

# Inputs:

Our input is an array of the expected return on investment for that perticular investment.

Example 1: Equal returns on each share

In [2]:
uniform_r = np.array([1,1,1,1,1,1,1,1,1,1])

Example 2: One investment is expected to make you rich, whilst all the others are expected to not do much

In [3]:
inv_r = np.array([1,1,1,1,1,1,1,1,1,100])

Example 3: A more realistic uniform random distribution between 0 and 2

In [4]:
uni_r = np.sort(np.random.uniform(0,10,20))

# Model:

What we want is to find the optimal portfolio distribution $\omega$ such that the both of the following are simultaneously maximised:

1. Entropy of the portfolio: $E(\omega) = \sum_{i = 1}^{N} - \omega_{i} \ln \omega_{i}$

2. Total expected return on investment: $R(\omega) = (I) \times \sum_{i = 1}^{N} \mu_{i} \omega_{i}$

By maximizing both, what we are therefore looking for is the $\omega$ for which given a constant $\mu$, it will minimize the following:

$$ L = \left( \sum_{i = 1}^{N} - \omega_{i} \ln \omega_{i}  \right)  -  \left( \sum_{i = 1}^{N} \mu_{i} \omega_{i} \right)$$

Given this rather simple optoimization function, we can now use SciPy optimization function to numericlly miniminize the above and find the optimal portfolio distribution according to our Entropy model.

In [5]:
def optimal(mu):
    
    N = len(mu)
    initial_guess = np.random.random(N)
    
    def f(x, arg):
        o = (1/sum(x)*x)
        return sum(o*np.log(o)) - sum(arg*o)
    
    
    ans = minimize(f, x0 = initial_guess, args = mu, bounds = Bounds(0.0001*np.ones_like(initial_guess), np.ones_like(initial_guess)))['x']
    
    omega = (1/sum(ans))*ans
    
    return (omega, mu)

# Outputs

### Example 1:
The expected result here is equal distribution of investment between all the shares

In [6]:
omega, mu = optimal(uniform_r)

In [7]:
omega

array([0.09999997, 0.10000001, 0.09999991, 0.0999989 , 0.10000057,
       0.0999999 , 0.09999989, 0.10000075, 0.10000012, 0.09999998])

As expected, our model suggets that we devide our investment evenly between all the shares.

### Example 2

The expected result is that our model will suggest us to invest most of our wealth into the share with the highest return on investment, whilst almost nothing on the other shares.

In [8]:
omega, mu = optimal(inv_r)

In [9]:
omega

array([9.99100809e-05, 9.99100809e-05, 9.99100809e-05, 9.99100809e-05,
       9.99100809e-05, 9.99100809e-05, 9.99100809e-05, 9.99100809e-05,
       9.99100809e-05, 9.99100809e-01])

As expected, most of the weight of our portfolio is on the one share with the disproportionate high expected return on investement.

### Example 3

The expected result is a more evenly distributed portfolio, however, shares with a higher return on investment will have most of the weight.

In [10]:
omega, mu = optimal(uni_r)

In [11]:
omega, mu

(array([9.17435369e-05, 1.80407457e-04, 2.06839567e-04, 2.10586843e-04,
        2.11584355e-04, 3.65245591e-04, 4.16381686e-04, 5.39198288e-04,
        6.82716997e-04, 8.68829586e-04, 1.03831569e-03, 1.72081195e-03,
        3.45497362e-03, 4.96024415e-03, 4.56885253e-02, 5.11965608e-02,
        1.22931508e-01, 2.52553605e-01, 2.55295187e-01, 2.57386735e-01]),
 array([1.95272478, 2.62872112, 2.76245954, 2.78329997, 2.78622917,
        3.33327891, 3.46445544, 3.72287005, 3.95871197, 4.19992989,
        4.38021672, 4.88275898, 5.57954738, 5.93961285, 8.16188883,
        8.27741825, 9.15104313, 9.87354937, 9.88099382, 9.89210402]))

As expected, the shares with highest expected return on investments hold the most amount of the invested wealth, decreasing as expected return decreases.